## Summary
1. Remove duplicated recorders
2. Check the missing values
3. Using frequency encoding for categorical variable
4. Explore the data imbalance between two classes.
5. Sample balance dataset from the mbalanced training data set 
6. Check the confusion matrix

## <span style="color:blue"> Throughout the exploration of the project, we can find the key challenge of it is the imbalance of data</span>


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
import xgboost as xgb

### Function for Frequency encoding of categorical variable

In [2]:
def freq_encode(pd_data, columns=False):
    '''Returns a DataFrame with encoded columns'''
    if columns==False:
        columns=list(pd_data.columns)
    encoded_cols = []
    nsamples = pd_data.shape[0]
    for col in columns:    
        freqs_cat = pd_data.groupby(col)[col].count()/nsamples
        encoded_col = pd_data[col].map(freqs_cat)
        encoded_col[encoded_col.isnull()] = np.nan
        encoded_cols.append(pd.DataFrame({'freq_'+str(col):encoded_col}))
    all_encoded = pd.concat(encoded_cols, axis=1)
    return all_encoded.loc[pd_data.index,:]

In [3]:
data_ =  pd.read_csv("data.csv")
data_.shape

(299285, 41)

### Remove duplicated rows

In [4]:
data = data_.drop_duplicates()
data.shape

(225014, 41)

### Check the missing values

In [5]:
for col in data.columns:
    total_null = data.loc[:, col].isnull().sum()
    print(col, 'has total nulls: ', total_null)

age has total nulls:  0
class_of_worker has total nulls:  0
detailed_industry_recode has total nulls:  0
detailed_occupation_recode has total nulls:  0
education has total nulls:  0
wage_per_hour has total nulls:  0
enroll_in_edu_inst_last_wk has total nulls:  0
marital_stat has total nulls:  0
major_industry_code has total nulls:  0
major_occupation_code has total nulls:  0
race has total nulls:  0
hispanic_origin has total nulls:  1219
sex has total nulls:  0
member_of_a_labor_union has total nulls:  0
reason_for_unemployment has total nulls:  0
full_or_part_time_employment_stat has total nulls:  0
capital_gains has total nulls:  0
capital_losses has total nulls:  0
dividends_from_stocks has total nulls:  0
tax_filer_stat has total nulls:  0
region_of_previous_residence has total nulls:  0
state_of_previous_residence has total nulls:  0
detailed_household_and_family_stat has total nulls:  0
detailed_household_summary_in_household has total nulls:  0
migration_code_change_in_msa has t

### By view the data, the missing values are marked as '?'

In [6]:
data.head() 

,age,class_of_worker,detailed_industry_recode,detailed_occupation_recode,education,wage_per_hour,enroll_in_edu_inst_last_wk,marital_stat,major_industry_code,major_occupation_code,...,country_of_birth_father,country_of_birth_mother,country_of_birth_self,citizenship,own_business_or_self_employed,fill_inc_questionnaire_for_veteran's_admin,veterans_benefits,weeks_worked_in_year,year,income_binned
0,73,Not in universe,0,0,High school graduate,0,Not in universe,Widowed,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,<=50k
1,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,<=50k
2,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,...,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,<=50k
3,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,<=50k
4,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,<=50k


### Mark the missing values as 'NaN'

In [7]:
data = data.replace('?', np.nan)

## Encode the categorical variables using frequecy encoding algorithm

In [8]:
xx = data.iloc[:,:data.shape[1]-1]
yy = data.iloc[:,-1].values

In [9]:
continuous_var = [0, 5, 16, 17, 18, 29, 38]
cat_var = list(set(np.arange(40)).difference(set(continuous_var)))

In [10]:
cat_columns = [xx.columns[i] for i in cat_var]
cat_encoded = freq_encode(xx, cat_columns)

In [11]:
contns = xx.iloc[:, continuous_var]
encoded_x = np.concatenate((cat_encoded.values, contns.values), axis=1)

In [12]:
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(yy)
encoded_y = label_encoder.transform(yy)

## Explore  balance of data

In [13]:
print(encoded_y[:20])
encoded_y.sum()/len(encoded_y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


0.0824082057116446

## Sample balance dataset from the mbalanced training data set

In [14]:
seed = 7
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(encoded_x, encoded_y, test_size=test_size, random_state=seed)

In [15]:
x_positive = X_train[y_train>0,:]
x_negative = X_train[y_train==0,:]
xy_positive = np.append(x_positive, np.ones((x_positive.shape[0],1)), axis=1)
xy_negative = np.append(x_negative, np.zeros((x_negative.shape[0],1)), axis=1)

In [16]:
neg_idx = np.random.choice(xy_negative.shape[0], xy_positive.shape[0], replace=False)
balanced_neg = xy_negative[neg_idx,:]

In [17]:
xy_train_ = np.vstack((xy_positive, balanced_neg))
xy_train = np.random.permutation(xy_train_)

In [18]:
X_train = xy_train[:, :(xy_train.shape[1]-1)]
y_train = xy_train[:,-1]

### Use XGBoost binary classifier

In [19]:
gbm = xgb.XGBClassifier(max_depth=5, n_estimators=300, learning_rate=0.05)
gbm.fit(X_train,y_train)
preds = gbm.predict(X_test)

## use the condition probability to measure the performace

In [20]:
confusion_mx = confusion_matrix(y_test, preds)
confusion_mx

array([[35154,  6153],
       [  466,  3230]])

In [21]:
print("Prob(positive on the condition positive) = %.4f"%(confusion_mx[0,0]/(confusion_mx[0,0]+confusion_mx[0,1])))
print("Prob(negative on the condition negative) = %.4f"%(confusion_mx[1,1]/(confusion_mx[1,1]+confusion_mx[1,0])))


Prob(positive on the condition positive) = 0.8510
Prob(negative on the condition negative) = 0.8739


### The performance with https://github.com/thanhluan001/Dataiku_Test/blob/master/USCensus.ipynb

In [22]:
print("Prob(negative on the condition negative) = %.4f"%(1343/(4843 +1343)))

Prob(negative on the condition negative) = 0.2171
